## Задание 1

Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.

In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 100 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Построение модели...
Процесс обучения...
Epoch 1/3
250/250 [==============================] - 38s 152ms/step - loss: 0.4674 - accuracy: 0.7705 - val_loss: 0.3772 - val_accuracy: 0.8306
Epoch 2/3
250/250 [==============================] - 37s 150ms/step - loss: 0.2907 - accuracy: 0.8796 - val_loss: 0.3636 - val_accuracy: 0.8387
Epoch 3/3
250/250 [==============================] - 5s 20ms/step - loss: 0.3755 - accuracy: 0.8399
Результат при тестировании: 0.37545573711395264
Тестовая точность: 0.8399199843406677


## Базовый вариант 

Результат при тестировании: 0.3741067051887512

Тестовая точность: 0.8390799760818481

In [ ]:
## Резюме по заданию 1

Эксперименты:

- при увеличении bacthsize для увеличения скорости обучения, loss сократился с 0.37 до 0.35
- при изменении функции optimizer c adam на SGD loss увеличился с 0.37 до 0.69
- пои измении функции optimizer c SGD на Nadam loss сократился до 0.35, а точность увеличилась до 0.84

Итого.

оптимальный выбор параметров показал преимущества функции optimizer Nadam, увеличения количества эпох до 3-х и порога dropout до 0.4. Все это привело к результату:

Результат при тестировании: 0.37545573711395264

Тестовая точность: 0.8399199843406677

Чтобы качественно улучшить результат, надо обратить внимание на предобработку текста. 

## Задача 2

Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [5]:
path = 'C:/Users/user/Documents/mydocs/neuron/data/'

In [6]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [7]:
# построчное чтение из примера с текстом 
with open(path + "London_Jack.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)

In [8]:
lines

[',        royallib.ru: http://royallib.ru',
 '  : http://royallib.ru/author/london_jack.html',
 '     : http://royallib.ru/book/london_jack/martin_eden.html',
 ' !',
 'martin eden',
 'by jack london',
 'chapter i',
 'the one opened the door with a latch-key and went in, followed by a young fellow who awkwardly removed his cap. he wore rough clothes that smacked of the sea, and he was manifestly out of place in the spacious hall in which he found himself. he did not know what to do with his cap, and was stuffing it into his coat pocket when the other took it from him. the act was done quietly and naturally, and the awkward young fellow appreciated it. "he understands," was his thought. "hell see me through all right."',
 'he walked at the others heels with a swing to his shoulders, and his legs spread unwittingly, as if the level floors were tilting up and sinking down to the heave and lunge of the sea. the wide rooms seemed too narrow for his rolling gait, and to himself he was in ter

In [9]:
text = " ".join(lines)

In [10]:
chars = set([c for c in text])
nb_chars = len(chars)

In [25]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 50, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 5 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    SimpleRNN(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
5969/5969 [==============================] - 54s 9ms/step - loss: 2.0733
Генерация из посева: ou forget," she interrupted, the quick surface of 
ou forget," she interrupted, the quick surface of the sack of the sack of the sack of the sack of the sack of the sack of the sack of the sack of the ==================================================
Итерация #: 1
5969/5969 [==============================] - 53s 9ms/step - loss: 1.8055
Генерация из посева: audlin he studied them, watching the beast rise an
audlin he studied them, watching the beast rise and so the sack of the sack of the sack of the sack of the sack of the sack of the sack of the sack of==================================================
Итерация #: 2
5969/5969 [==============================] - 53s 9ms/step - loss: 1.7064
Генерация из посева: flashed out to her and around her, in the way of h
flashed out to her and around her, in the way of his browned the strengers and the strengers and the strengers and the st

In [15]:
result_GRU = 'for the recognition i have received. that recogni for the recognition i have received. that recogning he was a mother and the part of the partion of the street and the concertate of the street and'

In [16]:
from googletrans import Translator
translator = Translator()
result_gru_ru = translator.translate(result_GRU, dest='ru')

In [19]:
print(result_gru_ru.text)

за признание, которое я получил. Это признание для признания, которое я получил. Признание, что он был матерью и частью части улицы и концерта улицы и


In [22]:
result_lstm = 'ent and kissed her, and each time her lips met his ent and kissed her, and each time her lips met his head and the street of the street of the street of the street of the street of the street of the st'

In [23]:
result_lstm_ru = translator.translate(result_lstm , dest='ru')

In [24]:
print(result_lstm_ru.text)

Энт и поцеловал ее, и каждый раз ее губы встретились с его энтом и поцеловал ее, и каждый раз ее губы встретились его головой и улицей улицы, улицей улицы улицы улицы


In [26]:
result_SimpleRNN = 's i saw the inhabitants i knew what id struck. ones i saw the inhabitants i knew what id struck. one that he had not the rear and the rear and the rear and the rear and the rear and the rear and the r'

In [28]:
result_SimpleRNN_ru = translator.translate(result_SimpleRNN, dest='ru')
print(result_SimpleRNN_ru.text) 

s я видел жителей, я знал, что поразило id. тех, кого я видел среди жителей, я знал, что поразило. тот, который у него был не тыл и тыл и тыл и тыл и тыл и тыл и т


In [ ]:
На данном этапе самый оптимальный вариант получился у алгоритма GRU. 